In [ ]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st 
from scipy.stats import linregress
import gmaps

#Import API key
from config import g_key


In [ ]:
#Save from CSVs
pk_go_csv = pd.read_csv("Resources/pkmn-go.csv")
poke_spawns = pd.read_csv("Resources/pokemon-spawns.csv")

merged_csv = pd.merge(pk_go_csv, poke_spawns, on="name")
merged_csv.head(None)
#Drop Unncessary Columns and Exact Duplicates
cleaned_csv = merged_csv.drop_duplicates()
cleaned_csv = cleaned_csv[['name', 'stamina', 'atk', 'def', 'capture_rate', 'spawn_chance', 'primary', 'cp', 'lat', 'lng']]
#Counts of Pokemon by Type
print(cleaned_csv.primary.value_counts())
cleaned_csv.reset_index(drop=True).head(None)




In [ ]:
#Breakdown of Counts by Species and Primary Type
poketype_df = cleaned_csv.groupby('primary')
poketype_df = pd.DataFrame(poketype_df['name'].value_counts())

poketype_df.head(50)

In [ ]:
# pie chart of distribution of pokemon types
#take "primary" column, create lists to plot
types = cleaned_csv["primary"].value_counts(dropna=False).keys().tolist()
counts = cleaned_csv["primary"].value_counts(dropna=False).tolist()

plt.pie(counts, labels=types, labeldistance=None)
plt.title("Pokemon by Type")
plt.axis('equal')
plt.savefig("Saved_Figures/pie_type")
plt.legend(types, loc="upper right", bbox_to_anchor=(1.2,1.025))
plt.show()

### Does attack level have a correlation to capture rate?

In [ ]:
#Capture Rate v Attack Level 
fig, ax = plt.subplots()

ax.scatter(cleaned_csv['capture_rate'], cleaned_csv["atk"])
plt.title('Capture rate vs. Attack level')
plt.xlabel('Capture Rate')
plt.ylabel('Attack Level')
plt.savefig("Saved_Figures/capture_v_attack")
plt.show()

## Linregress for Capture Rate vs Attack Level

In [ ]:
#Capture Rate v Attack Level with linregress line

#lin regress equation
x_data= cleaned_csv['capture_rate']
y_data= cleaned_csv["atk"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_data, y_data)
regress_values = x_data * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#plotting graph
plt.scatter(cleaned_csv['capture_rate'], cleaned_csv["atk"])
plt.plot(x_data, regress_values, "r-")
plt.annotate(line_eq,(.3,220), color='red')
plt.title('Capture rate vs. Attack level')
plt.xlabel('Capture Rate')
plt.ylabel('Attack Level')
plt.savefig("Saved_Figures/cature_v_attack_linreg")
print(f'The rvalue is: {rvalue}')
plt.show()


## Does capture rate have a correlation with Stamina levels?

In [ ]:
#Capture Rate v Stamina

plt.scatter(cleaned_csv['capture_rate'], cleaned_csv["stamina"])
plt.title('Capture rate vs. Stamina')
plt.xlabel('Capture Rate')
plt.ylabel('Stamina')
plt.savefig("Saved_Figures/capture_v_stamina")
plt.show()

In [ ]:
#Capture Rate v Stamina with linregress line

#lin regress equation
x_data2= cleaned_csv['capture_rate']
y_data2= cleaned_csv["stamina"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_data2, y_data2)
regress_values = x_data2 * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#plotting graph
plt.scatter(cleaned_csv['capture_rate'], cleaned_csv["stamina"])
plt.plot(x_data, regress_values, "r-")
plt.annotate(line_eq,(.3,400), color='red')
plt.title('Capture rate vs. Stamina')
plt.xlabel('Capture Rate')
plt.ylabel('Stamina')
plt.savefig("Saved_Figures/capture_v_stamina_linreg")
print(f'The rvalue is: {rvalue}')
plt.show()

In [ ]:
#Capture Rate v Defense

plt.scatter(cleaned_csv['capture_rate'], cleaned_csv["def"])
plt.title('Capture rate vs. Defense')
plt.xlabel('Capture Rate')
plt.ylabel('Defense')
plt.savefig("Saved_Figures/capture_v_defense")
plt.show()

In [ ]:
#Capture Rate v Defense with linregress line

#lin regress equation
x_data3= cleaned_csv['capture_rate']
y_data3= cleaned_csv["def"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_data3, y_data3)
regress_values = x_data3 * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#plotting graph
plt.scatter(cleaned_csv['capture_rate'], cleaned_csv["def"])
plt.plot(x_data, regress_values, "r-")
plt.annotate(line_eq,(.3,200), color='red')
plt.title('Capture rate vs. Defense')
plt.xlabel('Capture Rate')
plt.ylabel('Defense')
plt.savefig("Saved_Figures/capture_v_defense_linreg")
print(f'The rvalue is: {rvalue}')
plt.show()


In [ ]:
poketype = cleaned_csv.groupby('primary')
avg_spawn_chance = poketype['spawn_chance'].mean()
avg_cp = poketype['cp'].mean()
avg_capture = poketype['capture_rate'].mean()
#spawn_df = spawn_chance.rename_axis('Primary').to_frame('Average Spawn Chance')
#spawn_df.sort_values(by='Average Spawn Chance', ascending = False)
avg_table = pd.DataFrame({
    "Average CP": avg_cp,
    "Average Spawn Chance": avg_spawn_chance,
    "Average Capture Rate": avg_capture,
    
})

print("Summary of Average Capture Related Stats")
avg_table

In [ ]:
#Bar Graph of Average CP
avg_table['Average CP'].plot(kind='bar', figsize=(10,7))
plt.title("Average CP by Type")
plt.xlabel("Primary Pokemon Type")
plt.ylabel("Combat Power")
plt.show()

In [ ]:
#Bar Graph of Average Spawn Chance
avg_table['Average Spawn Chance'].plot(kind='bar', figsize=(10,7))
plt.title("Average Spawn Chance by Type")
plt.xlabel("Primary Pokemon Type")
plt.ylabel("Spawn Chance")
plt.show()

In [ ]:
#Bar Graph of Average Capture Rate
avg_table['Average Capture Rate'].plot(kind='bar', figsize=(10,7))
plt.title("Average Capture Rate by Type")
plt.xlabel("Primary Pokemon Type")
plt.ylabel("Capture Rate")
plt.show()


## Use gmaps to visualize relationship between geographical features and pokemon spawn locations, saving map images to folder, and transfer to powerpoint

In [ ]:


#Access maps with API key
gmaps.configure(api_key=g_key)

#Ask for user input to search map by Pokemon Type for heatmap visualization
query = str(input("Enter Pokemon Type"))
highest_avg_cp = cleaned_csv[cleaned_csv['primary'] == query]
locations = highest_avg_cp[["lat", "lng"]]
cp = highest_avg_cp["cp"].astype(float)
# Using the template add marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>CP</dt><dd>{cp}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
poke_info = [info_box_template.format(**row) for index, row in highest_avg_cp.iterrows()]

#Plot Heatmap
fig = gmaps.figure()
#Add markers
markers = gmaps.marker_layer(locations, info_box_content=poke_info)
fig.add_layer(markers)
#Create Heatlayer
heat_layer = gmaps.heatmap_layer(locations, cp, dissipating=False, point_radius=1)

#Add layer
fig.add_layer(heat_layer)

fig